In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('TimeML_TEST_zero_shot_round2_withcommas.csv')
# 'withcommas' means that the csv is comma-separated and not e.g. tab-separated
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...


In [3]:
def split_negation(sentence):
    sentence = sentence.replace("couldn't <O>\n", "could <O>\nn't <O>\n")
    sentence = sentence.replace("shouldn't <O>\n", "should <O>\nn't <O>\n")
    sentence = sentence.replace("wouldn't <O>\n", "would <O>\nn't <O>\n")
    sentence = sentence.replace("don't <O>\n", "do <O>\nn't<O>\n")
    sentence = sentence.replace("doesn't <O>\n", "does <O>\nn't <O>\n")
    sentence = sentence.replace("isn't <O>\n", "is <O>\nn't <O>\n")
    sentence = sentence.replace("aren't <O>\n", "are <O>\nn't <O>\n")
    sentence = sentence.replace("didn't <O>\n", "did <O>\nn't <O>\n")
    sentence = sentence.replace("wasn't <O>\n", "was <O>\nn't <O>\n")
    sentence = sentence.replace("weren't <O>\n", "were <O>\nn't <O>\n")
    sentence = sentence.replace("hasn't <O>\n", "has <O>\nn't <O>\n")
    sentence = sentence.replace("haven't <O>\n", "have <O>\nn't <O>\n")
    sentence = sentence.replace("hadn't <O>\n", "had <O>\nn't <O>\n")
    if "\n's <O>\n" in sentence:
        sentence = sentence.replace("\n's <O>\n", "\n's <O>\n")
    else:
        sentence = sentence.replace("'s <O>\n", " <O>\n's <O>\n")
    return sentence

In [4]:
df['LLM_response_split_negation'] = df['LLM_response'].apply(split_negation)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...


In [5]:
print(df['LLM_response'][0])

A <O>
Cyprus <O>
exit <O>
from <O>
the <O>
euro <O>
union <O>
if <O>
it <O>
comes <O>
to <O>
that <O>
would <O>
have <O>
a <O>
devastating <O>
effect <O>
on <O>
the <O>
country's <O>
citizens <O>
who <O>
are <O>
among <O>
the <O>
most <O>
indebted <O>
in <O>
the <O>
euro <O>
zone <O>.


In [6]:
print(df['LLM_response_split_negation'][0])

A <O>
Cyprus <O>
exit <O>
from <O>
the <O>
euro <O>
union <O>
if <O>
it <O>
comes <O>
to <O>
that <O>
would <O>
have <O>
a <O>
devastating <O>
effect <O>
on <O>
the <O>
country <O>
's <O>
citizens <O>
who <O>
are <O>
among <O>
the <O>
most <O>
indebted <O>
in <O>
the <O>
euro <O>
zone <O>.


In [7]:
def token_tag(annotated_sentence):
    lines = annotated_sentence.split('\n')
    list_of_token_tag = []
    for line in lines:
        line = line.split()
        if len(line) == 2:
            if 'EVENT' in line[1] or 'O' in line[1]:
                list_of_token_tag.append(line)
                list_of_token_tag
    tuples = []
    for pair in list_of_token_tag:
        tuples.append(tuple(pair))
    return tuple(tuples) # I am using tuples for further processing, see zip()

In [8]:
df['list_of_token_tag_pairs'] = df['LLM_response_split_negation'].apply(token_tag)

In [9]:
(df['list_of_token_tag_pairs'] == ()).sum()

19

In [10]:
df_errors = df.loc[df['list_of_token_tag_pairs'] == (), :].reset_index()
df_errors.head()

,index,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs
0,8,8,140128564756224,`` There have been too many bailouts in Europe...,Given the sentence between the brackets <`` Th...,There\n<EVENT>\nhave\n<EVENT>\nbeen\n<EVENT>\n...,There\n<EVENT>\nhave\n<EVENT>\nbeen\n<EVENT>\n...,()
1,9,9,140128564014576,"`` This is not a Lehman, '' he said, referring...",Given the sentence between the brackets <`` Th...,This\n<EVENT>\nis\n<O>\nnot\n<O>\na\n<O>\nLehm...,This\n<EVENT>\nis\n<O>\nnot\n<O>\na\n<O>\nLehm...,()
2,30,30,140128563042128,At the top of the app is a text entry field th...,Given the sentence between the brackets <At th...,At <EVENT> the <EVENT> top <O> of <O> the <O> ...,At <EVENT> the <EVENT> top <O> of <O> the <O> ...,()
3,39,39,140128563332688,The prospect of Keep incorporating features of...,Given the sentence between the brackets <The p...,The \nEVENT\nprospect\nEVENT\nof\nEVENT\nKeep\...,The \nEVENT\nprospect\nEVENT\nof\nEVENT\nKeep\...,()
4,48,48,140128564015728,"In the U.S., 11 companies were selected last y...",Given the sentence between the brackets <In th...,"In\n<EVENT>\nthe\n<O>\nU.S.,\n<O>\n11\n<O>\nco...","In\n<EVENT>\nthe\n<O>\nU.S.,\n<O>\n11\n<O>\nco...",()


In [11]:
def error_type(annotated_sentence):
    lines = annotated_sentence.split('\n')
    for line in lines:
        line = line.split()
        if len(line) < 2:
            return 'each token and each tag is in a separate line'
        else:
            return 'the whole sentence is in one continuous line'
             # it might be better to say that there are multiple tokens in one line
    return

In [12]:
df_errors['error_type'] = df_errors['LLM_response_split_negation'].apply(error_type)
df_errors.head()

,index,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,error_type
0,8,8,140128564756224,`` There have been too many bailouts in Europe...,Given the sentence between the brackets <`` Th...,There\n<EVENT>\nhave\n<EVENT>\nbeen\n<EVENT>\n...,There\n<EVENT>\nhave\n<EVENT>\nbeen\n<EVENT>\n...,(),each token and each tag is in a separate line
1,9,9,140128564014576,"`` This is not a Lehman, '' he said, referring...",Given the sentence between the brackets <`` Th...,This\n<EVENT>\nis\n<O>\nnot\n<O>\na\n<O>\nLehm...,This\n<EVENT>\nis\n<O>\nnot\n<O>\na\n<O>\nLehm...,(),each token and each tag is in a separate line
2,30,30,140128563042128,At the top of the app is a text entry field th...,Given the sentence between the brackets <At th...,At <EVENT> the <EVENT> top <O> of <O> the <O> ...,At <EVENT> the <EVENT> top <O> of <O> the <O> ...,(),the whole sentence is in one continuous line
3,39,39,140128563332688,The prospect of Keep incorporating features of...,Given the sentence between the brackets <The p...,The \nEVENT\nprospect\nEVENT\nof\nEVENT\nKeep\...,The \nEVENT\nprospect\nEVENT\nof\nEVENT\nKeep\...,(),each token and each tag is in a separate line
4,48,48,140128564015728,"In the U.S., 11 companies were selected last y...",Given the sentence between the brackets <In th...,"In\n<EVENT>\nthe\n<O>\nU.S.,\n<O>\n11\n<O>\nco...","In\n<EVENT>\nthe\n<O>\nU.S.,\n<O>\n11\n<O>\nco...",(),each token and each tag is in a separate line


In [13]:
df_errors['error_type'].value_counts(normalize = True)

each token and each tag is in a separate line    0.684211
the whole sentence is in one continuous line     0.315789
Name: error_type, dtype: float64

In [14]:
with open('test_tags_adjusted_TimeML.txt', encoding = 'utf8') as infile:
    gold_TimeML_TEST = infile.read()
print(gold_TimeML_TEST[:100])


A	<O>
Cyprus	<O>
exit	<EVENT>
from	<O>
the	<O>
euro	<O>
union	<O>
,	<O>
if	<O>
it	<O>
comes	<EVENT>


In [15]:
gold_TimeML_TEST = gold_TimeML_TEST.split('\n\n')

In [16]:
df2 = pd.DataFrame({'gold_TimeML_TEST': gold_TimeML_TEST})
df2

,gold_TimeML_TEST
0,\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...
1,And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...
2,ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...
3,"And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n..."
4,Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...
...,...
270,Memorial\t<O>\nsays\t<EVENT>\ninspectors\t<O>\...
271,A\t<O>\nstatement\t<O>\non\t<O>\nthe\t<O>\nMem...
272,Memorial\t<O>\ndirector\t<O>\nArseny\t<O>\nRog...
273,He\t<O>\ninsisted\t<EVENT>\nthat\t<O>\nthe\t<O...


In [17]:
df['gold_TimeML_TEST'] = df2['gold_TimeML_TEST']
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...


In [18]:
df['gold_list_of_token_tag_pairs'] = df['gold_TimeML_TEST'].apply(token_tag)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr..."
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni..."
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ..."
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have..."


In [19]:
# remove commas and fullstops from tuple lists to make the postprocessing feasible
def remove_commas_fullstops(tuple_of_tuples):
    list_of_tuples = list(tuple_of_tuples)
    if len(tuple_of_tuples) !=0:
        for item in list_of_tuples:
            list_without_commas_and_fullstops = [item for item in list_of_tuples if item[0] != ',' and item[0] != '.']
        return tuple(list_without_commas_and_fullstops)
    else:
        return tuple_of_tuples
    return    

In [20]:
df['list_of_token_tag_pairs_without_commas_fullstops'] = df['list_of_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ..."
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ..."
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ..."
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have..."


In [21]:
df['gold_list_of_token_tag_pairs_without_commas_fullstops'] = df['gold_list_of_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr..."
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni..."
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ..."
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have..."


In [22]:
equal_length = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    x = len(a) == len(b)
    equal_length.append(x)   

In [23]:
print(equal_length[:50])

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, True, False, False, False, False, True, False, True, False, True, False, False, False, False, False]


In [24]:
df['equal_length']= pd.DataFrame({
    "equal_length": equal_length
})
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...",False
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (...",False
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...",False


In [25]:
df['equal_length'].value_counts()

False    242
True      33
Name: equal_length, dtype: int64

In [26]:
#shorter than gold
shorter_than_gold = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    y = len(a) < len(b)
    shorter_than_gold.append(y)

df['shorter_than_gold']= pd.DataFrame({
    "shorter_than_gold": shorter_than_gold
})
df.head()

df['shorter_than_gold'].value_counts()

True     229
False     46
Name: shorter_than_gold, dtype: int64

In [27]:
#longer than gold
longer_than_gold = []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs']
    b=row['gold_list_of_token_tag_pairs']
    z = len(a) > len(b)
    longer_than_gold.append(z)

df['longer_than_gold']= pd.DataFrame({
    "longer_than_gold": longer_than_gold
})
df.head()

df['longer_than_gold'].value_counts()

False    262
True      13
Name: longer_than_gold, dtype: int64

In [28]:
#equal length without commas and fullstops
equal_length_without_commas_fullstops= []
for idx, row in df.iloc[0:].iterrows():
    a=row['list_of_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    x = len(a) == len(b)
    equal_length_without_commas_fullstops.append(x)

In [29]:
df['equal_length_without_commas_fullstops']= pd.DataFrame({
    "equal_length_without_commas_fullstops": equal_length_without_commas_fullstops
})
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False,True,False,True
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False,True,False,True
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...",False,True,False,True
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (...",False,True,False,True
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...",False,True,False,True


In [30]:
df['equal_length_without_commas_fullstops'].value_counts()

False    150
True     125
Name: equal_length_without_commas_fullstops, dtype: int64

In [31]:
complete_dataset_for_eval= []
for idx, row in df.iloc[0:].iterrows():
    a=row['equal_length_without_commas_fullstops']
    b=row['LLM_response_split_negation']
    c=row['gold_TimeML_TEST']
    if a == True:
        complete_dataset_for_eval.append(b)
    else:
        complete_dataset_for_eval.append(c)

df['complete_dataset_for_eval'] = complete_dataset_for_eval

In [32]:
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False,True,False,True,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False,True,False,True,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...",False,True,False,True,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (...",False,True,False,True,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...",False,True,False,True,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...


In [33]:
# where equal_length_without_commas_fullstops is False, change EVENT to O
complete_dataset_for_eval_adjusted= []
for idx, row in df.iloc[0:].iterrows():
    a=row['equal_length_without_commas_fullstops']
    b=row['complete_dataset_for_eval']
    if a == False:
        o_tags = b.replace("<EVENT>", "<O>")
        complete_dataset_for_eval_adjusted.append(o_tags)
    else:
        complete_dataset_for_eval_adjusted.append(b)

df['complete_dataset_for_eval_adjusted'] = complete_dataset_for_eval_adjusted
df.head(2)

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False,True,False,True,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False,True,False,True,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...


In [34]:
print(df['complete_dataset_for_eval_adjusted'][9])

``	<O>
This	<O>
is	<O>
not	<O>
a	<O>
Lehman	<O>
,	<O>
''	<O>
he	<O>
said	<O>
,	<O>
referring	<O>
to	<O>
the	<O>
disastrous	<O>
chain	<O>
reaction	<O>
touched	<O>
off	<O>
by	<O>
the	<O>
collapse	<O>
of	<O>
Lehman	<O>
Brothers	<O>
in	<O>
2008	<O>
.	<O>


In [35]:
print(df['complete_dataset_for_eval'][9])

``	<O>
This	<O>
is	<O>
not	<O>
a	<O>
Lehman	<O>
,	<O>
''	<O>
he	<O>
said	<EVENT>
,	<O>
referring	<EVENT>
to	<O>
the	<O>
disastrous	<O>
chain	<O>
reaction	<EVENT>
touched	<EVENT>
off	<O>
by	<O>
the	<O>
collapse	<EVENT>
of	<O>
Lehman	<O>
Brothers	<O>
in	<O>
2008	<O>
.	<O>


In [36]:
df.shape

(275, 17)

In [37]:
# list of token tag pairs
df['for_eval_token_tag_pairs'] = df['complete_dataset_for_eval_adjusted'].apply(token_tag)

In [38]:
# remove commas and fullstops
df['for_eval_token_tag_pairs_without_commas_fullstops'] = df['for_eval_token_tag_pairs'].apply(remove_commas_fullstops)
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted,for_eval_token_tag_pairs,for_eval_token_tag_pairs_without_commas_fullstops
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False,True,False,True,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ..."
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False,True,False,True,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <EVENT>), (for, <O>), (European, <O>), ..."
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...",False,True,False,True,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ..."
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (...",False,True,False,True,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <EVENT>), (that, <O>), (economists, <EV..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...",False,True,False,True,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have..."


In [39]:
# equal length without commas and fullstops
equal_length_without_commas_fullstops= []
for idx, row in df.iloc[0:].iterrows():
    a=row['for_eval_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    x = len(a) == len(b)
    equal_length_without_commas_fullstops.append(x)

In [40]:
# check equal length one more time
equal_length_without_commas_fullstops.count(True)

275

In [41]:
equal_length_without_commas_fullstops.count(False)

0

In [42]:
zippedlines = []
for idx, row in df.iloc[0:].iterrows():    
    a=row['for_eval_token_tag_pairs_without_commas_fullstops']
    b=row['gold_list_of_token_tag_pairs_without_commas_fullstops']
    a= list(a)
    b= list(b)
    zippedlines.append(list(zip(b, a)))

In [43]:
df['zippedlines']= pd.DataFrame({
    "zippedlines": zippedlines
})
df

,Unnamed: 0,id,sentence,prompt,LLM_response,LLM_response_split_negation,list_of_token_tag_pairs,gold_TimeML_TEST,gold_list_of_token_tag_pairs,list_of_token_tag_pairs_without_commas_fullstops,gold_list_of_token_tag_pairs_without_commas_fullstops,equal_length,shorter_than_gold,longer_than_gold,equal_length_without_commas_fullstops,complete_dataset_for_eval,complete_dataset_for_eval_adjusted,for_eval_token_tag_pairs,for_eval_token_tag_pairs_without_commas_fullstops,zippedlines
0,0,140128563332016,"A Cyprus exit from the euro union, if it come...",Given the sentence between the brackets < A Cy...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...",\nA\t<O>\nCyprus\t<O>\nexit\t<EVENT>\nfrom\t<O...,"((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <EVENT>), (fr...",False,True,False,True,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,A <O>\nCyprus <O>\nexit <O>\nfrom <O>\nthe <O>...,"((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","((A, <O>), (Cyprus, <O>), (exit, <O>), (from, ...","[((A, <O>), (A, <O>)), ((Cyprus, <O>), (Cyprus..."
1,1,140128563041488,"And for European unity and diplomacy, the Cypr...",Given the sentence between the brackets <And f...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...",And\t<O>\nfor\t<O>\nEuropean\t<O>\nunity\t<O>\...,"((And, <O>), (for, <O>), (European, <O>), (uni...","((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <O>), (for, <O>), (European, <O>), (uni...",False,True,False,True,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,And <EVENT>\nfor <O>\nEuropean <O>\nunity <O>\...,"((And, <EVENT>), (for, <O>), (European, <O>), ...","((And, <EVENT>), (for, <O>), (European, <O>), ...","[((And, <O>), (And, <EVENT>)), ((for, <O>), (f..."
2,2,140128562949760,"ut for the broader financial system in Europe,...",Given the sentence between the brackets <ut fo...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...",ut\t<O>\nfor\t<O>\nthe\t<O>\nbroader\t<O>\nfin...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...",False,True,False,True,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,ut <O>\nfor <O>\nthe <O>\nbroader <O>\nfinanci...,"((ut, <O>), (for, <O>), (the, <O>), (broader, ...","((ut, <O>), (for, <O>), (the, <O>), (broader, ...","[((ut, <O>), (ut, <O>)), ((for, <O>), (for, <O..."
3,3,140128563272656,"And that, economists and investors contend, is...",Given the sentence between the brackets <And t...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","And\t<O>\nthat\t<O>\n,\t<O>\neconomists\t<O>\n...","((And, <O>), (that, <O>), (,, <O>), (economist...","((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <O>), (that, <O>), (economists, <O>), (...",False,True,False,True,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,And <EVENT>\nthat <O>\neconomists <EVENT>\nand...,"((And, <EVENT>), (that, <O>), (economists, <EV...","((And, <EVENT>), (that, <O>), (economists, <EV...","[((And, <O>), (And, <EVENT>)), ((that, <O>), (..."
4,4,140128563331792,Greece may well have been too big to fail last...,Given the sentence between the brackets <Greec...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,Greece <O>\nmay <O>\nwell <O>\nhave <O>\nbeen ...,"((Greece, <O>), (may, <O>), (well, <O>), (have...",Greece\t<O>\nmay\t<O>\nwell\t<O>\nhave\t<O>\nb...,"((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...","((Greece, <O>), (may, <O>), (well, <O>), (have...",False,True

In [44]:
df['zippedlines'][0]

[(('A', '<O>'), ('A', '<O>')),
 (('Cyprus', '<O>'), ('Cyprus', '<O>')),
 (('exit', '<EVENT>'), ('exit', '<O>')),
 (('from', '<O>'), ('from', '<O>')),
 (('the', '<O>'), ('the', '<O>')),
 (('euro', '<O>'), ('euro', '<O>')),
 (('union', '<O>'), ('union', '<O>')),
 (('if', '<O>'), ('if', '<O>')),
 (('it', '<O>'), ('it', '<O>')),
 (('comes', '<EVENT>'), ('comes', '<O>')),
 (('to', '<O>'), ('to', '<O>')),
 (('that', '<O>'), ('that', '<O>')),
 (('would', '<O>'), ('would', '<O>')),
 (('have', '<O>'), ('have', '<O>')),
 (('a', '<O>'), ('a', '<O>')),
 (('devastating', '<O>'), ('devastating', '<O>')),
 (('effect', '<EVENT>'), ('effect', '<O>')),
 (('on', '<O>'), ('on', '<O>')),
 (('the', '<O>'), ('the', '<O>')),
 (('country', '<O>'), ('country', '<O>')),
 (("'s", '<O>'), ("'s", '<O>')),
 (('citizens', '<O>'), ('citizens', '<O>')),
 (('who', '<O>'), ('who', '<O>')),
 (('are', '<O>'), ('are', '<O>')),
 (('among', '<O>'), ('among', '<O>')),
 (('the', '<O>'), ('the', '<O>')),
 (('most', '<O>'), ('mos

In [45]:
def zips(tuples):
    lines = []
    for item in tuples:
        lines.append(f'{id(item)}\t{item[0][0]}\t{item[0][1]}\t{item[1][0]}\t{item[1][1]}\n')
    return ''.join(lines)

In [46]:
for idx, row in df.iloc[0:].iterrows():    
    a=row['zippedlines']
    a= zips(a)
    with open('TimeML_zero_shot_gold_predicted_1706.txt', 'a', encoding='utf8') as outfile:
        outfile.write(f'{a}\n')    

In [47]:
with open('TimeML_zero_shot_gold_predicted_1706.txt', encoding='utf8') as infile:
    text = infile.read()

In [48]:
print(text[:2000])

1198319819456	A	<O>	A	<O>
1198319825792	Cyprus	<O>	Cyprus	<O>
1198319819008	exit	<EVENT>	exit	<O>
1198319818624	from	<O>	from	<O>
1198319818752	the	<O>	the	<O>
1198319823552	euro	<O>	euro	<O>
1198319823232	union	<O>	union	<O>
1198319822400	if	<O>	if	<O>
1198319823040	it	<O>	it	<O>
1198319821952	comes	<EVENT>	comes	<O>
1198319821248	to	<O>	to	<O>
1198319821056	that	<O>	that	<O>
1198319820288	would	<O>	would	<O>
1198319818816	have	<O>	have	<O>
1198319821504	a	<O>	a	<O>
1198319777472	devastating	<O>	devastating	<O>
1198319818048	effect	<EVENT>	effect	<O>
1198319819648	on	<O>	on	<O>
1198319777728	the	<O>	the	<O>
1198319818944	country	<O>	country	<O>
1198318302848	's	<O>	's	<O>
1198319825600	citizens	<O>	citizens	<O>
1198319818688	who	<O>	who	<O>
1198319779648	are	<O>	are	<O>
1198318328640	among	<O>	among	<O>
1198319819968	the	<O>	the	<O>
1198319825664	most	<O>	most	<O>
1198318378688	indebted	<EVENT>	indebted	<O>
1198319777600	in	<O>	in	<O>
1198319821376	the	<O>	the	<O>
1198319822912	euro	<

In [49]:
text = text.split('\n')

In [50]:
lines = []
for item in text:
    line = item.split()
    lines.append(line)
    
print(lines[:20])

[['1198319819456', 'A', '<O>', 'A', '<O>'], ['1198319825792', 'Cyprus', '<O>', 'Cyprus', '<O>'], ['1198319819008', 'exit', '<EVENT>', 'exit', '<O>'], ['1198319818624', 'from', '<O>', 'from', '<O>'], ['1198319818752', 'the', '<O>', 'the', '<O>'], ['1198319823552', 'euro', '<O>', 'euro', '<O>'], ['1198319823232', 'union', '<O>', 'union', '<O>'], ['1198319822400', 'if', '<O>', 'if', '<O>'], ['1198319823040', 'it', '<O>', 'it', '<O>'], ['1198319821952', 'comes', '<EVENT>', 'comes', '<O>'], ['1198319821248', 'to', '<O>', 'to', '<O>'], ['1198319821056', 'that', '<O>', 'that', '<O>'], ['1198319820288', 'would', '<O>', 'would', '<O>'], ['1198319818816', 'have', '<O>', 'have', '<O>'], ['1198319821504', 'a', '<O>', 'a', '<O>'], ['1198319777472', 'devastating', '<O>', 'devastating', '<O>'], ['1198319818048', 'effect', '<EVENT>', 'effect', '<O>'], ['1198319819648', 'on', '<O>', 'on', '<O>'], ['1198319777728', 'the', '<O>', 'the', '<O>'], ['1198319818944', 'country', '<O>', 'country', '<O>']]


In [51]:
ids = []
for line in lines:
    if len(line) ==5 and line[1] != line[3]:
        ids.append([line[0], line[1], line[3]])
    ids
    
print(ids)

[['1198319825536', '-LRB-', '-'], ['1198319826880', '-RRB-', '-'], ['1198319867904', 'companies', 'and'], ['1198319867968', 'and', 'companies'], ['1198319868032', 'governments', 'and'], ['1198319868096', 'to', 'governments'], ['1198319868160', 'make', 'to'], ['1198319868224', 'good', 'make'], ['1198319868288', 'on', 'good'], ['1198319868352', 'them', 'on'], ['1198319868416', 'the', 'them'], ['1198319868480', 'view', 'the'], ['1198319868544', 'is', 'view'], ['1198319868608', 'forming', 'is'], ['1198319868672', 'in', 'forming'], ['1198319868736', 'Berlin', 'in'], ['1198319868800', 'and', 'Berlin'], ['1198319868864', 'Brussels', 'and'], ['1198319868928', 'that', 'Brussels'], ['1198319868992', 'a', 'that'], ['1198319869056', 'signal', 'a'], ['1198319869120', 'must', 'signal'], ['1198319869184', 'be', 'must'], ['1198319869248', 'sent', 'be'], ['1198319869312', 'that', 'sent'], ['1198319869376', 'citizens', 'that'], ['1198319869440', 'and', 'citizens'], ['1198319869504', 'investors', 'and'],

In [52]:
len(ids)

112

In [53]:
for item in ids:
    print(f'{item[0]}\t{item[1]}\t{item[2]}')

1198319825536	-LRB-	-
1198319826880	-RRB-	-
1198319867904	companies	and
1198319867968	and	companies
1198319868032	governments	and
1198319868096	to	governments
1198319868160	make	to
1198319868224	good	make
1198319868288	on	good
1198319868352	them	on
1198319868416	the	them
1198319868480	view	the
1198319868544	is	view
1198319868608	forming	is
1198319868672	in	forming
1198319868736	Berlin	in
1198319868800	and	Berlin
1198319868864	Brussels	and
1198319868928	that	Brussels
1198319868992	a	that
1198319869056	signal	a
1198319869120	must	signal
1198319869184	be	must
1198319869248	sent	be
1198319869312	that	sent
1198319869376	citizens	that
1198319869440	and	citizens
1198319869504	investors	and
1198319869568	must	investors
1198319869632	start	must
1198319869696	accepting	start
1198319869760	losses	accepting
1198319869824	for	losses
1198319869888	the	for
1198319869952	euro	the
1198319870016	zone	euro
1198319870080	to	zone
1198319870144	survive	to
1198319870208	in	survive
1198319870272	the	in
119831